In [ ]:
!pip install pypsa

In [ ]:
import pypsa
import numpy as np
import gc
import pandas as pd
import os
import pickle
from google.colab import drive
import xarray as xr # Import xarray here

# Define a simple, picklable representation for linear expressions
class PicklableLinearExpr:
    def __init__(self, vars, coeffs, const=0):
        self.vars = vars
        self.coeffs = coeffs
        self.const = const

    def __repr__(self):
        terms = [f"{coeff}*var_id_{var}" for var, coeff in zip(self.vars, self.coeffs)]
        expr_str = " + ".join(terms)
        if self.const != 0:
            expr_str += f" + {self.const}"
        return expr_str

def fix_artificial_lines_hardcoded(network):
    """
    Fix artificial lines with hardcoded values to match existing non-extendable lines:
    - s_nom = 442.4 (your desired capacity)
    - s_nom_extendable = False
    - s_nom_min = 0
    - s_nom_max = inf
    - extendable = False (if column exists)
    """
    print("=== FIXING ARTIFICIAL LINES WITH HARDCODED VALUES ===")
    fixed_capacity=442.4

    # Find artificial lines
    artificial_lines = [line for line in network.lines.index
                       if any(keyword in str(line).lower() for keyword in ['new', '<->', 'artificial'])]

    print(f"Found {len(artificial_lines)} artificial lines to fix:")

    # Fix each artificial line with hardcoded values
    for line_name in artificial_lines:
        print(f"\n🔧 Fixing: {line_name}")

        # s_nom = 442.4 (your fixed capacity)
        old_s_nom = network.lines.loc[line_name, 's_nom']
        network.lines.loc[line_name, 's_nom'] = fixed_capacity
        print(f"    s_nom: {old_s_nom} → {fixed_capacity}")

        # s_nom_extendable = False
        if 's_nom_extendable' not in network.lines.columns:
            network.lines['s_nom_extendable'] = False
        network.lines.loc[line_name, 's_nom_extendable'] = False
        print(f"    s_nom_extendable: → False")

        # s_nom_min = 0
        if 's_nom_min' not in network.lines.columns:
            network.lines['s_nom_min'] = 0.0
        network.lines.loc[line_name, 's_nom_min'] = 0.0
        print(f"    s_nom_min: → 0.0")

        # s_nom_max = inf
        if 's_nom_max' not in network.lines.columns:
            network.lines['s_nom_max'] = float('inf')
        network.lines.loc[line_name, 's_nom_max'] = float('inf')
        print(f"    s_nom_max: → inf")

    return network

def create_pypsa_network(network_file):
    """Create a PyPSA network from the .nc file."""
    # Initialize network
    network = pypsa.Network(network_file)
    for storage_name in network.storage_units.index:
        # Use .loc for direct assignment to avoid SettingWithCopyWarning
        network.storage_units.loc[storage_name, 'cyclic_state_of_charge'] = False

    fix_artificial_lines_hardcoded(network)

    return network

def _variable_mapping(network_file):
    """
    Create a mapping from variable IDs to variable names.

    Parameters:
    -----------
    network_file : str
        Path to the PyPSA network file

    Returns:
    --------
    dict : var_id_to_name - Mapping from variable IDs to variable names
    """
    print("Starting variable ID mapping")
    network = create_pypsa_network(network_file)
    # Create model once - this is an expensive operation
    temp_model = network.optimize.create_model()

    # Create variable ID to name mapping
    var_id_to_name = {}
    for var_name, variable in temp_model.variables.items():
        # Get the variable labels (IDs) for this variable
        var_labels = variable.labels

        if hasattr(var_labels, 'values'):
            # Multi-dimensional variable
            labels_flat = var_labels.values.flatten()
            coords = variable.labels.coords
            for i, label in enumerate(labels_flat):
                if label != -1:  # -1 means no variable
                    # Create a name that includes the index for multi-dim variables
                    if len(coords) > 0:
                        # Get the coordinate values for this flat index
                        unravel_idx = np.unravel_index(i, var_labels.shape)
                        coord_values = []
                        for dim_idx, dim_name in enumerate(var_labels.dims):
                            coord_val = coords[dim_name].values[unravel_idx[dim_idx]]

                            # Handle datetime64 values properly
                            if isinstance(coord_val, np.datetime64) or hasattr(coord_val, 'strftime'):
                                # Convert datetime to string in ISO format
                                try:
                                    coord_val = pd.Timestamp(coord_val).isoformat()
                                except:
                                    # Fallback if conversion fails
                                    coord_val = str(coord_val)

                            coord_values.append(f"{dim_name}={coord_val}")

                        full_name = f"{var_name}[{','.join(coord_values)}]"
                    else:
                        full_name = f"{var_name}[{i}]"
                    var_id_to_name[label] = full_name
        else:
            # Scalar variable
            var_id_to_name[var_labels] = var_name

    print(f"Finished variable ID mapping, found {len(var_id_to_name)} variables")

    # Clean up to free memory
    del temp_model
    gc.collect()

    return var_id_to_name

def save_var_mapping(var_id_to_name, network_file, output_dir="var_constraint_map"):
    """
    Save the variable ID to name mapping to a file.

    Parameters:
    -----------
    var_id_to_name : dict
        Mapping from variable IDs to variable names
    network_file : str
        Path to the network file used to create the mapping
    output_dir : str, optional
        Directory to save the mapping file to

    Returns:
    --------
    str : Path to the saved mapping file
    """
    # Handle Google Drive if needed
    if '/content/drive' in output_dir:
        try:
            from google.colab import drive
            drive.mount('/content/drive')
        except ImportError:
            pass  # Not in Colab environment

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Get the network filename without path or extension
    network_name = os.path.basename(network_file)
    network_name = os.path.splitext(network_name)[0]

    # Create output filename
    var_map_file = os.path.join(output_dir, f"{network_name}_var_id_to_name.pkl")

    # Save mapping to file
    with open(var_map_file, 'wb') as f:
        pickle.dump(var_id_to_name, f)

    print(f"Saved variable mapping to: {var_map_file}")

    return var_map_file

def load_var_mapping(network_file, input_dir="var_constraint_map"):
    """
    Load previously saved variable ID to name mapping from a file.

    Parameters:
    -----------
    network_file : str
        Path to the network file used to create the mapping
    input_dir : str, optional
        Directory where the mapping file is stored

    Returns:
    --------
    dict : var_id_to_name - The loaded variable ID to name mapping
    """
    # Handle Google Drive if needed
    if '/content/drive' in input_dir:
        try:
            from google.colab import drive
            drive.mount('/content/drive')
        except ImportError:
            pass  # Not in Colab environment

    # Get the network filename without path or extension
    network_name = os.path.basename(network_file)
    network_name = os.path.splitext(network_name)[0]

    # Create input filename
    var_map_file = os.path.join(input_dir, f"{network_name}_var_id_to_name.pkl")

    # Check if file exists
    if not os.path.exists(var_map_file):
        raise FileNotFoundError(f"Variable mapping file for {network_name} not found in {input_dir}")

    # Load mapping from file
    with open(var_map_file, 'rb') as f:
        var_id_to_name = pickle.load(f)

    print(f"Loaded variable mapping from: {var_map_file}")

    return var_id_to_name

In [ ]:
def _evaluate_stored_objective(self):
        """
        Direct evaluation without mock objects.
        Only includes terms from the current snapshot in the objective function.
        """
        self.var_id_to_name= load_mappings(network_file=self.network_file, input_dir=self.input_dir)
        # Get variable name mapping for current network
        id_to_name = self.var_id_to_name

        # Get the current snapshot name
        current_snapshot = self.network.snapshots[self.snapshot_idx]

        # Get current variable values
        variable_names = []
        var_indices = []
        vars_flat = self.objective_vars.values.flatten()
        coeffs_flat = self.objective_coeffs.values.flatten()

        # Filter variables to only include those from the current snapshot
        for i, var_id in enumerate(vars_flat):
            if var_id != -1 and var_id in id_to_name:
                var_name = id_to_name[var_id]

                # Check if this variable belongs to the current snapshot
                if 'snapshot=' in var_name:
                    # Extract the snapshot value from the variable name
                    snapshot_part = var_name.split('snapshot=')[1].split(',')[0].split(']')[0]
                    if snapshot_part == str(current_snapshot):
                        variable_names.append(var_name)
                        var_indices.append(i)
                else:
                    # Include variables without snapshot dimension (like investment variables)
                    variable_names.append(var_name)
                    var_indices.append(i)

        var_values = self.create_variable_values_mapping(variable_names)
        ipdb.set_trace()

        # Direct mathematical evaluation using only variables from current snapshot
        if var_indices:
            result = np.sum(coeffs_flat[var_indices] *
                        [var_values.get(name, 0) for name in variable_names]) + \
                    self.objective_const
        else:
            # If no variables for this snapshot, just return the constant
            result = self.objective_const

        return result

In [ ]:
def main():
    # Example usage
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        network_file = "/content/drive/MyDrive/Colab_Notebooks/networks_1_year_connected/elec_s_10_ec_lc1.0_1h.nc"
    except ImportError:
        # Not in Colab environment, use local path
        network_file = "path/to/your/local/network_file.nc"  # Update this to your local path

    # Define output directory
    output_dir = "/content/drive/MyDrive/Colab_Notebooks/var_constraint_map"

    # Create variable ID to name mapping
    var_id_to_name = _variable_mapping(network_file)

    # Save mapping
    var_map_file = save_var_mapping(var_id_to_name, network_file, output_dir)

    print(f"Created variable ID to name mapping with {len(var_id_to_name)} entries")
    print(f"Saved variable mapping to {var_map_file}")

    # Optional: Verify loading works
    try:
        loaded_mapping = load_var_mapping(network_file, output_dir)
        print(f"Successfully verified loading with {len(loaded_mapping)} entries")
    except Exception as e:
        print(f"Error verifying load: {e}")

In [ ]:
main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Starting variable ID mapping


Index(['0', '1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5', '6',
       '7', '8', '9', 'lines new ZA0 4 <-> ZA2 0 AC',
       'lines new ZA0 0 <-> ZA1 0 AC', 'lines new ZA0 0 <-> ZA3 0 AC'],
      dtype='object', name='Line')
Index(['ZA0 0', 'ZA0 1', 'ZA0 2', 'ZA0 3', 'ZA0 4', 'ZA0 5', 'ZA0 6', 'ZA0 7',
       'ZA0 8', 'ZA0 9', 'ZA1 0', 'ZA2 0', 'ZA3 0'],
      dtype='object', name='Bus')


=== FIXING ARTIFICIAL LINES WITH HARDCODED VALUES ===
Found 3 artificial lines to fix:

🔧 Fixing: lines new ZA0 4 <-> ZA2 0 AC
    s_nom: 0.0 → 442.4
    s_nom_extendable: → False
    s_nom_min: → 0.0
    s_nom_max: → inf

🔧 Fixing: lines new ZA0 0 <-> ZA1 0 AC
    s_nom: 0.0 → 442.4
    s_nom_extendable: → False
    s_nom_min: → 0.0
    s_nom_max: → inf

🔧 Fixing: lines new ZA0 0 <-> ZA3 0 AC
    s_nom: 0.0 → 442.4
    s_nom_extendable: → False
    s_nom_min: → 0.0
    s_nom_max: → inf


/usr/local/lib/python3.12/dist-packages/linopy/expressions.py:1861: FutureWarning:

In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): '_term' ('_term',) The recommendation is to set join explicitly for this case.



Finished variable ID mapping, found 735840 variables
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved variable mapping to: /content/drive/MyDrive/Colab_Notebooks/var_constraint_map/elec_s_10_ec_lc1.0_1h_var_id_to_name.pkl
Created variable ID to name mapping with 735840 entries
Saved variable mapping to /content/drive/MyDrive/Colab_Notebooks/var_constraint_map/elec_s_10_ec_lc1.0_1h_var_id_to_name.pkl
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded variable mapping from: /content/drive/MyDrive/Colab_Notebooks/var_constraint_map/elec_s_10_ec_lc1.0_1h_var_id_to_name.pkl
Successfully verified loading with 735840 entries


In [ ]:
# drive.mount('/content/drive')
# network_file = "/content/drive/MyDrive/Colab_Notebooks/networks_1_year_connected/elec_s_10_ec_lc1.0_1h.nc"
# network = create_pypsa_network(network_file)
# # Create model once - this is an expensive operation
# temp_model = network.optimize.create_model()

# for name, constraint_group in temp_model.constraints.items():
#     print(name)

In [ ]:
# network.lines['s_max_pu']